Importing Libraries

In [66]:
pip install scikit-learn-extra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
from IPython.display import Audio, display
import pandas as pd
import soundfile as sf
import os
from os import listdir
from os.path import isfile, join
import shutil
from google.colab import drive
import copy
import sklearn
from sklearn.preprocessing import minmax_scale
import numpy as np
import json
import librosa
import librosa.display
from numpy.lib.utils import source
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.impute import SimpleImputer
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler , normalize
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import SpectralClustering
from sklearn_extra.cluster import KMedoids
import matplotlib.cm as cm
from sklearn import metrics

Loading data and cleansing bad datas

In [68]:
drive.mount('/content/gdrive')
dataset1 = pd.read_csv('/content/gdrive/My Drive/ML/Final Project/Audio_Dataset.csv')
dataset2 = pd.read_csv('/content/gdrive/My Drive/ML/Final Project/dataV2.csv')
voice_names_str = dataset1.song_name.values
X_total = dataset1.iloc[:, 1:].values
emotions = dataset2.emotionID.values
gender = dataset2.sex.values
sentences=dataset2.textID.values
voice_names_total = np.array([int(x[:-4]) for x in voice_names_str])
voice_id = dataset2['voice id'].values
X = []
voice_names = []
for i, name in enumerate(voice_names_total):
  if name in voice_id:
    X.append(X_total[i, :])
    voice_names.append(name)

X = np.array(X)
voice_names = np.array(voice_names)


y_emotions = []
y_gender_str = []
y_sentences=[]
for i, name in enumerate(voice_names):
  y_emotions.append(emotions[np.where(voice_id == name)][0])
  y_gender_str.append(gender[np.where(voice_id == name)][0].lower())
  y_sentences.append(sentences[np.where(voice_id == name)][0])

y_emotions = np.array(y_emotions)
y_sentences=np.array( y_sentences)
drive.mount('/content/gdrive')
dataset1 = pd.read_csv('/content/gdrive/My Drive/ML/Final Project/Audio_Dataset.csv')
dataset2 = pd.read_csv('/content/gdrive/My Drive/ML/Final Project/dataV2.csv')
y_gender_str = np.array(y_gender_str)
y_gender = np.zeros(y_gender_str.shape)
y_gender[y_gender_str == 'f'] = 1

idx_to_delete = np.where(y_emotions > 4)
X = np.delete(X, idx_to_delete, axis = 0)
y_emotions = np.delete(y_emotions, idx_to_delete, axis = 0)
y_gender = np.delete(y_gender, idx_to_delete, axis = 0)
y_sentences=np.delete(y_sentences, idx_to_delete, axis = 0)
idx_to_delete = np.where(y_sentences== 0)
X = np.delete(X, idx_to_delete, axis = 0)
y_emotions = np.delete(y_emotions, idx_to_delete, axis = 0)
y_gender = np.delete(y_gender, idx_to_delete, axis = 0)
y_sentences=np.delete(y_sentences, idx_to_delete, axis = 0)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X)
X = imputer.transform(X)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


scaling feature vector

In [69]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)

we have 10 different sentences with 4 type of emotions so each participant has approximately 40 different voices so all in all number of participants is approximately equal to number of all voices to 40


In [70]:
participant_no=len(X)/(10*4)

**Clustering using K-Means method**

In [71]:
kmeans_1 = KMeans(init="random",n_clusters=2,n_init=10,max_iter=300,random_state=42)
kmeans_1.fit(scaled_features)
kmeans_2 = KMeans(init="random",n_clusters=4,n_init=10,max_iter=300,random_state=42)
kmeans_2.fit(scaled_features)
kmeans_3 = KMeans(init="random",n_clusters=10,n_init=10,max_iter=300,random_state=42)
kmeans_3.fit(scaled_features)
kmeans_4 = KMeans(init="random",n_clusters=int(np.floor(participant_no)),n_init=10,max_iter=300,random_state=42)
kmeans_4.fit(scaled_features)

KMeans(init='random', n_clusters=399, random_state=42)

**Clustering using K-Medoids Clustering method**

In [72]:
kmedoids_1 = KMedoids(n_clusters=2, random_state=42)
kmedoids_1.fit(scaled_features)
kmedoids_2 = KMedoids(n_clusters=4, random_state=42)
kmedoids_2.fit(scaled_features)
kmedoids_3 = KMedoids(n_clusters=10, random_state=42)
kmedoids_3.fit(scaled_features)
kmedoids_4 = KMedoids(n_clusters=int(np.floor(participant_no)), random_state=0)
kmedoids_4.fit(scaled_features)

/usr/local/lib/python3.7/dist-packages/sklearn_extra/cluster/_k_medoids.py:279: UserWarning: Cluster 306 is empty! self.labels_[self.medoid_indices_[306]] may not be labeled with its corresponding cluster (306).
  "its corresponding cluster ({k}).".format(k=k)


KMedoids(n_clusters=399, random_state=0)

Calculating Gender error for both k-means and k-medoids method


In [73]:
gender_error_kmeans=100*sum(y_gender-kmeans_1.labels_)/len(X)
gender_error_kmedoids=100*sum(y_gender-kmedoids_1.labels_)/len(X)

Calculating emotion error for both k-means and k-medoids method


In [74]:
y_emotion_cluster_kmeans=kmeans_2.labels_+1
y_emotion_cluster_kmedoids=kmedoids_2.labels_+1
total_emotions=np.unique(y_emotions)
real_emotions=np.zeros(len(total_emotions), dtype=int)
clustered_emotions_kmeans=np.zeros(len(total_emotions), dtype=int)
clustered_emotions_kmedoids=np.zeros(len(total_emotions), dtype=int)
for i,y in enumerate(y_emotions):
  for k,j in enumerate(total_emotions):
    if(y==j):
     real_emotions[k]+=1 
for i,y in enumerate(y_emotion_cluster_kmeans):
 for k,j in enumerate(total_emotions):
    if(y==j):
     clustered_emotions_kmeans[k]+=1
for i,y in enumerate(y_emotion_cluster_kmedoids):
 for k,j in enumerate(total_emotions):
    if(y==j):
     clustered_emotions_kmedoids[k]+=1 
er_kmeans=real_emotions-clustered_emotions_kmeans
er_kmedoids=real_emotions-clustered_emotions_kmedoids
emotions_error_kmeans=0
emotions_error_kmedoids=0
for i in er_kmeans:
  if(i>0):
    emotions_error_kmeans+=i
for i in er_kmedoids:
  if(i>0):
    emotions_error_kmedoids+=i  
emotions_error_kmeans=100*emotions_error_kmeans/len(X)
emotions_error_kmedoids=100*emotions_error_kmedoids/len(X)

Calculating sentences error for both k-means and k-medoids method


In [75]:
y_sentences_cluster_kmeans=kmeans_3.labels_+1
y_sentences_cluster_kmedoids=kmedoids_3.labels_+1
total_sentences=np.unique(y_sentences)
real_sentences=np.zeros(len(total_sentences), dtype=int)
clustered_sentences_kmeans=np.zeros(len(total_sentences), dtype=int)
clustered_sentences_kmedoids=np.zeros(len(total_sentences), dtype=int)
for i,y in enumerate(y_sentences):
  for k,j in enumerate(total_sentences):
    if(y==j):
     real_sentences[k]+=1 
for i,y in enumerate(y_emotion_cluster_kmeans):
 for k,j in enumerate(total_sentences):
    if(y==j):
     clustered_sentences_kmeans[k]+=1
for i,y in enumerate(y_emotion_cluster_kmedoids):
 for k,j in enumerate(total_sentences):
    if(y==j):
     clustered_sentences_kmedoids[k]+=1 
er_kmeans=real_sentences-clustered_sentences_kmeans
er_kmedoids=real_sentences-clustered_sentences_kmedoids
sentences_error_kmeans=0
sentences_error_kmedoids=0
for i in er_kmeans:
  if(i>0):
    sentences_error_kmeans+=i
for i in er_kmedoids:
  if(i>0):
    sentences_error_kmedoids+=i 
sentences_error_kmeans=100*sentences_error_kmeans/len(X)
sentences_error_kmedoids=100*sentences_error_kmedoids/len(X)      

Calculating participants error for both k-means and k-medoids method


In [76]:
y_participants_kmeans=kmeans_4.labels_+1
y_participants_kmedoids=kmedoids_4.labels_+1
total_participants=np.unique(y_participants_kmeans)
real_participants=40*np.ones(len(total_participants),dtype=int)
clustered_participants_kmeans=np.zeros(len(total_participants), dtype=int)
clustered_participants_kmedoids=np.zeros(len(total_participants), dtype=int)
for i,y in enumerate(y_participants_kmeans):
 for k,j in enumerate(total_participants):
    if(y==j):
     clustered_participants_kmeans[k]+=1
for i,y in enumerate(y_participants_kmedoids):
 for k,j in enumerate(total_participants):
    if(y==j):
     clustered_participants_kmedoids[k]+=1     
er_kmeans=real_participants-clustered_participants_kmeans
er_kmedoids=real_participants-clustered_participants_kmedoids
participants_error_kmeans=0
participants_error_kmedoids=0
for i in er_kmeans:
  if(i>0):
    participants_error_kmeans+=i
for i in er_kmedoids:
  if(i>0):
    participants_error_kmedoids+=i
participants_error_kmeans=100*participants_error_kmeans/len(X)
participants_error_kmedoids=100*participants_error_kmedoids/len(X)    

In [77]:
from tabulate import tabulate

In [78]:
table_1=[['type of error','K-means','K-medoids'],
       ['error in gender',gender_error_kmeans,gender_error_kmedoids],
       ['error in emotions',emotions_error_kmeans,emotions_error_kmedoids],
       ['error in sentences',sentences_error_kmeans,sentences_error_kmedoids],
       ['error in participants',participants_error_kmeans,participants_error_kmedoids]]

error table: number of wrong prediction(clustering with label) to total number of length of feature vector in percents

In [79]:
print(tabulate(table_1, headers='firstrow', tablefmt='fancy_grid'))

╒═══════════════════════╤═══════════╤═════════════╕
│ type of error         │   K-means │   K-medoids │
╞═══════════════════════╪═══════════╪═════════════╡
│ error in gender       │   8.93907 │     1.67647 │
├───────────────────────┼───────────┼─────────────┤
│ error in emotions     │   8.93282 │     4.1724  │
├───────────────────────┼───────────┼─────────────┤
│ error in sentences    │  59.5021  │    59.5021  │
├───────────────────────┼───────────┼─────────────┤
│ error in participants │  16.8773  │    32.053   │
╘═══════════════════════╧═══════════╧═════════════╛


calculating Silhoutte score and Calinski Harabaz Index for both methods and all clusters

In [80]:
cluster_labels_kmeans_1 = kmeans_1.fit_predict(X)
silhouette_avg_kmeans_1 = silhouette_score(X, cluster_labels_kmeans_1)
calinski_indx_kmeans_1=metrics.calinski_harabasz_score(X, cluster_labels_kmeans_1)
cluster_labels_kmedoids_1 = kmedoids_1.fit_predict(X)
silhouette_avg_kmedoids_1 = silhouette_score(X, cluster_labels_kmedoids_1)
calinski_indx_kmedoids_1=metrics.calinski_harabasz_score(X, cluster_labels_kmedoids_1)

cluster_labels_kmeans_2 = kmeans_2.fit_predict(X)
silhouette_avg_kmeans_2 = silhouette_score(X, cluster_labels_kmeans_2)
calinski_indx_kmeans_2=metrics.calinski_harabasz_score(X, cluster_labels_kmeans_2)
cluster_labels_kmedoids_2 = kmedoids_2.fit_predict(X)
silhouette_avg_kmedoids_2 = silhouette_score(X, cluster_labels_kmedoids_2)
calinski_indx_kmedoids_2=metrics.calinski_harabasz_score(X, cluster_labels_kmedoids_2)

cluster_labels_kmeans_3 = kmeans_3.fit_predict(X)
silhouette_avg_kmeans_3 = silhouette_score(X, cluster_labels_kmeans_3)
calinski_indx_kmeans_3=metrics.calinski_harabasz_score(X, cluster_labels_kmeans_3)
cluster_labels_kmedoids_3 = kmedoids_3.fit_predict(X)
silhouette_avg_kmedoids_3 = silhouette_score(X, cluster_labels_kmedoids_3)
calinski_indx_kmedoids_3=metrics.calinski_harabasz_score(X, cluster_labels_kmedoids_3)

cluster_labels_kmeans_4 = kmeans_4.fit_predict(X)
silhouette_avg_kmeans_4 = silhouette_score(X, cluster_labels_kmeans_4)
calinski_indx_kmeans_4=metrics.calinski_harabasz_score(X, cluster_labels_kmeans_4)
cluster_labels_kmedoids_4 = kmedoids_4.fit_predict(X)
silhouette_avg_kmedoids_4 = silhouette_score(X, cluster_labels_kmedoids_4)
calinski_indx_kmedoids_4=metrics.calinski_harabasz_score(X, cluster_labels_kmedoids_4)

In [81]:
table_2=[['number of clusters','K-means','K-medoids'],
       ['n_cluster=2',silhouette_avg_kmeans_1,silhouette_avg_kmedoids_1],
       ['n_cluster=4',silhouette_avg_kmeans_2,silhouette_avg_kmedoids_2],
       ['n_cluster=10',silhouette_avg_kmeans_3,silhouette_avg_kmedoids_3],
       ['n_cluster=total num of participants',silhouette_avg_kmeans_4,silhouette_avg_kmedoids_4]]

Silhoutte score table 

In [82]:
print(tabulate(table_2, headers='firstrow', tablefmt='fancy_grid'))

╒═════════════════════════════════════╤═══════════╤═════════════╕
│ number of clusters                  │   K-means │   K-medoids │
╞═════════════════════════════════════╪═══════════╪═════════════╡
│ n_cluster=2                         │  0.412599 │   0.403558  │
├─────────────────────────────────────┼───────────┼─────────────┤
│ n_cluster=4                         │  0.290588 │   0.256881  │
├─────────────────────────────────────┼───────────┼─────────────┤
│ n_cluster=10                        │  0.235705 │   0.190277  │
├─────────────────────────────────────┼───────────┼─────────────┤
│ n_cluster=total num of participants │  0.129132 │   0.0722907 │
╘═════════════════════════════════════╧═══════════╧═════════════╛


In [83]:
table_3=[['number of clusters','K-means','K-medoids'],
       ['n_cluster=2',calinski_indx_kmeans_1,calinski_indx_kmedoids_1],
       ['n_cluster=4',calinski_indx_kmeans_2,calinski_indx_kmedoids_2],
       ['n_cluster=10',calinski_indx_kmeans_3,calinski_indx_kmedoids_3],
       ['n_cluster=total num of participants',calinski_indx_kmeans_4,calinski_indx_kmedoids_4]]

Calinski score table

In [84]:
print(tabulate(table_3, headers='firstrow', tablefmt='fancy_grid'))

╒═════════════════════════════════════╤═══════════╤═════════════╕
│ number of clusters                  │   K-means │   K-medoids │
╞═════════════════════════════════════╪═══════════╪═════════════╡
│ n_cluster=2                         │  14951.5  │   14804.7   │
├─────────────────────────────────────┼───────────┼─────────────┤
│ n_cluster=4                         │  12145.2  │   11256.2   │
├─────────────────────────────────────┼───────────┼─────────────┤
│ n_cluster=10                        │   7892.7  │    6828.08  │
├─────────────────────────────────────┼───────────┼─────────────┤
│ n_cluster=total num of participants │   1342.58 │     393.878 │
╘═════════════════════════════════════╧═══════════╧═════════════╛
